## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Medical Reasoning LLM - 14B]()
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

- **Model**: `JSL-Medical-Reasoning-LLM-14B`
- **Model Description**: Medical model for summarization, question answering (open-book and closed-book), and general chat.

In [ ]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import os
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [ ]:
model_name = "JSL-Medical-Reasoning-LLM-14B"

real_time_inference_instance_type = "ml.g4dn.12xlarge"
batch_transform_inference_instance_type = "ml.g4dn.12xlarge"

## 2. Create a deployable model from the model package.

In [30]:
# Define ModelPackage
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn, 
    sagemaker_session=sagemaker_session, 
)

## Model Configuration Documentation  

### Default Configuration  
The container comes with the following default configurations:  

| Parameter                  | Default Value | Description                                                                   |  
|----------------------------|---------------|-------------------------------------------------------------------------------|  
| **`dtype`**                | `float16`     | Data type for model weights and activations                                   |  
| **`max_model_len`**        | `32,768`      | Maximum length for input and output combined (`input + output ≤ max_model_len`) |  
| **`tensor_parallel_size`** | Auto          | Automatically set to the number of available GPUs                            |  
| **`host`**                 | `0.0.0.0`     | Host name                                                                     |  
| **`port`**                 | `8080`        | Port number                                                                   |  

### Hardcoded Settings  
The following settings are hardcoded in the container and cannot be changed:  

| Parameter       | Value           | Description                           |  
|-----------------|-----------------|---------------------------------------|  
| **`model`**     | `/opt/ml/model` | Model path where SageMaker mounts the model |  

### Configurable Environment Variables  
You can customize the vLLM server by setting environment variables when creating the model.  

**Any parameter from the [vLLM documentation](https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html#vllm-serve) can be set using the corresponding environment variable with the `SM_VLLM_` prefix.**  

The container uses a script similar to the [SageMaker entrypoint example](https://docs.vllm.ai/en/latest/examples/sagemaker_entrypoint.html) from the vLLM documentation to convert environment variables to command-line arguments.  

---  

## Input Format  

### 1. Chat Completion  

#### Example Payload  
```json  
{  
    "model": "/opt/ml/model",  
    "messages": [  
        {"role": "system", "content": "You are a helpful medical assistant."},  
        {"role": "user", "content": "What should I do if I have a fever and body aches?"}  
    ],  
    "max_tokens": 1024,  
    "temperature": 0.7  
}  
```  

For additional parameters:  
- [ChatCompletionRequest](https://github.com/vllm-project/vllm/blob/v0.7.3/vllm/entrypoints/openai/protocol.py#L212)  
- [OpenAI's Chat API](https://platform.openai.com/docs/api-reference/chat/create)  

---  

### 2. Text Completion  

#### Single Prompt Example  
```json  
{  
    "model": "/opt/ml/model",  
    "prompt": "How can I maintain good kidney health?",  
    "max_tokens": 512,  
    "temperature": 0.6  
}  
```  

#### Multiple Prompts Example  
```json  
{  
    "model": "/opt/ml/model",  
    "prompt": [  
        "How can I maintain good kidney health?",  
        "What are the best practices for kidney care?"  
    ],  
    "max_tokens": 512,  
    "temperature": 0.6  
}  
```  

Reference:  
- [CompletionRequest](https://github.com/vllm-project/vllm/blob/v0.7.3/vllm/entrypoints/openai/protocol.py#L642)  
- [OpenAI's Completions API](https://platform.openai.com/docs/api-reference/completions/create)  

---  

### Important Notes:
- **Streaming Responses:** Add `"stream": true` to your request payload to enable streaming
- **Model Path Requirement:** Always set `"model": "/opt/ml/model"` (SageMaker's fixed model location)

## 3. Create an SageMaker Endpoint

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
    model_data_download_timeout=3600
)

### 3.1 Real-time inference via Amazon SageMaker Endpoint

#### Initial setup

In [41]:
prompt1 = """A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus.

Which of the following is the best treatment for this patient?
A: Ampicillin
B: Ceftriaxone
C: Ciprofloxacin
D: Doxycycline
E: Nitrofurantoin
"""

prompt2 = "What should I do if I have a fever and body aches?"

prompts = [
    "How can I maintain good kidney health?",
    "What are the symptoms of high blood pressure?"
]

In [42]:
import time

def invoke_realtime_endpoint(record):

    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType="application/json",
        Accept="application/json",
        Body=json.dumps(record),
    )

    return json.load(response["Body"])

#### Chat Completion

In [43]:
input_data = {
    "model": "/opt/ml/model",
    "messages": [
        {"role": "system", "content": "You are a helpful medical assistant."},
        {"role": "user", "content": prompt1},
    ],
    "max_tokens": 2048,
    "temperature": 0.7,
}

result = invoke_realtime_endpoint(input_data)
output_content = result['choices'][0]['message']['content']
print(output_content)

The patient presents with symptoms of a urinary tract infection (UTI) during pregnancy, specifically burning upon urination, which has not improved despite home remedies. Given her gestational age (22 weeks) and the absence of systemic symptoms like fever, the most appropriate first-line treatment for an uncomplicated UTI in pregnancy is an antibiotic that is both safe for the mother and the fetus.

**Ampicillin** (Option A) is typically used for treating infections such as UTIs in pregnant women, but it is not the most appropriate choice here because it is no longer often recommended as a first-line treatment due to increasing resistance. Additionally, it may not cover the common pathogens responsible for UTIs in pregnant women.

**Ciprofloxacin** (Option C) and **Doxycycline** (Option D) are **contraindicated** in pregnancy due to potential risks to the fetus, including joint abnormalities with doxycycline and cartilage toxicity with ciprofloxacin.

**Ceftriaxone** (Option B) is a br

#### Text Completion

In [45]:
input_data ={
        "model": "/opt/ml/model",
        "prompt": prompt2,
        "max_tokens": 2048,
        "temperature": 0.7,
    }

result = invoke_realtime_endpoint(input_data)
output_text = result['choices'][0]['text']
print(output_text)

 If you have a fever and body aches, here are some steps to take:

1. **Rest**: Get plenty of rest to help your body fight off the illness.

2. **Stay Hydrated**: Drink plenty of fluids like water, clear broths, or fruit juices to maintain your hydration levels.

3. **Take Medications**: Over-the-counter medications such as acetaminophen (Tylenol) or ibuprofen (Advil) can help reduce fever and alleviate body aches. Make sure to follow the recommended dosages and consult a healthcare provider if you have any underlying health conditions or are taking other medications.

4. **Use Fever-Reducing Measures**: Cool compresses or a lukewarm bath can help lower a high fever.

5. **Monitor Your Symptoms**: Keep an eye on any changes in your condition. If your symptoms worsen, persist for more than a few days, or if you experience additional concerning symptoms (such as difficulty breathing, persistent vomiting, severe headache, or rash), seek medical care promptly.

6. **Practice Good Hygiene**

### 3.2 Real-time inference response as a stream via Amazon SageMaker Endpoint

In [47]:
def invoke_streaming_endpoint(record):
    try:
        response = sm_runtime.invoke_endpoint_with_response_stream(
            EndpointName=model_name,
            Body=json.dumps(record),
            ContentType="application/json",
            Accept="text/event-stream"
        )

        for event in response["Body"]:
            if "PayloadPart" in event:
                chunk = event["PayloadPart"]["Bytes"].decode("utf-8")
                if chunk.startswith("data:"):
                    try:
                        data = json.loads(chunk[5:].strip())
                        if "choices" in data and len(data["choices"]) > 0:
                            choice = data["choices"][0]
                            if "text" in choice:
                                yield choice["text"]
                            elif "delta" in choice and "content" in choice["delta"]:
                                yield choice["delta"]["content"]

                    except json.JSONDecodeError:
                        continue 
            elif "ModelStreamError" in event:
                error = event["ModelStreamError"]
                yield f"\nStream error: {error['Message']} (Error code: {error['ErrorCode']})"
                break
            elif "InternalStreamFailure" in event:
                failure = event["InternalStreamFailure"]
                yield f"\nInternal stream failure: {failure['Message']}"
                break
    except Exception as e:
        yield f"\nAn error occurred during streaming: {str(e)}"

#### Chat Completion

In [48]:
payload = {
    "model": "/opt/ml/model",
    "messages": [
        {"role": "system", "content": "You are a helpful medical assistant."},
        {"role": "user", "content": prompt1}
    ],
    "max_tokens": 2048,
    "temperature": 0.7,
    "stream": True
}

for chunk in invoke_streaming_endpoint(payload):
    print(chunk, end="", flush=True)

The most appropriate treatment for this patient is **E: Nitrofurantoin**.

**Explanation:**
- **Clinical Presentation:** The patient presents with symptoms of a urinary tract infection (UTI), specifically dysuria (burning during urination), which has been worsening despite increased fluid intake and cranberry extract use.
- **Pregnancy Considerations:** Pregnant women require special antibiotic considerations due to potential risks to the fetus. Specifically, antibiotics like fluoroquinolones (Ciprofloxacin) and tetracyclines (Doxycycline) are contraindicated in pregnant women as they can cause harm to the developing fetus.
- **Drug Options:**
  - **Ampicillin:** While ampicillin is generally safe in pregnancy, it is less commonly used for uncomplicated UTIs and may not be the first-line choice.
  - **Ceftriaxone:** This is typically reserved for more severe infections or complicated UTIs. It is generally considered safe in pregnancy but is not the first-line treatment for uncomplicate

#### Text Completion

In [50]:
payload = {
    "model": "/opt/ml/model",
    "prompt": prompt2,
    "max_tokens": 2048,
    "temperature": 0.7,
    "stream": True
}

for chunk in invoke_streaming_endpoint(payload):
    print(chunk, end="", flush=True)

 If you have a fever and body aches, here are some steps you can take to manage your symptoms:

1. **Stay Hydrated**: Drink plenty of fluids like water, herbal teas, or clear broths. Staying hydrated helps your body fight off infections and can help lower a fever.

2. **Rest**: Allow your body the time it needs to recover. Resting helps conserve your energy and supports the immune response.

3. **Take Medication**: Over-the-counter medications like acetaminophen (Tylenol) or ibuprofen (Advil, Motrin) can help reduce fever and alleviate body aches. Follow the dosing instructions carefully and consult a provider if you have any concerns about which medication to use.

4. **Use Fever-Reducing Measures**: A cool bath or sponge bath can help bring down a fever. Avoid using alcohol or cold water, as this can cause shivering and increase your body temperature.

5. **Comfort Measures**: Dress in lightweight clothing and use light blankets to avoid overheating. Use a fan or air to maintain a co

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [56]:
import json
import os

# JSON file names
validation_json_file_name1 = "input1.json"


# JSON paths
validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/"


def write_and_upload_to_s3(input_data, file_name):
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_name}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [57]:
input_json_data1 = json.dumps(
    {
        "model": "/opt/ml/model",
        "prompt": prompts,
        "max_tokens": 2048,
        "temperature": 0.7,
    }
)

write_and_upload_to_s3(input_json_data1, f"{validation_json_file_name1}")

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path,
)
transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [ ]:
from urllib.parse import urlparse

def retrieve_json_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [61]:
retrieve_json_output_from_s3(validation_json_file_name1)

{'id': 'cmpl-86c6f7fe2ead4dc79ba5942eecfb9930',
 'object': 'text_completion',
 'created': 1743489812,
 'model': '/opt/ml/model',
 'choices': [{'index': 0,
   'text': " Maintaining good kidney health is essential because kidneys play a critical role in filtering waste products and excess water from your blood, regulating blood pressure, and producing hormones that help keep your bones strong. Here are some tips to help you maintain kidney health:\n\n1. **Stay Hydrated**: Drink plenty of water throughout the day to help your kidneys remove toxins and waste products from your blood.\n\n2. **Eat a Balanced Diet**: Include plenty of whole grains, fruits, and vegetables in your diet. Cut back on salt and processed foods to help manage blood pressure, which is a major risk factor for kidney disease.\n\n3. **Maintain a Healthy Weight**: Being overweight or obese can increase your risk of developing kidney disease. Regular exercise and a healthy diet can help you maintain a healthy weight.\n\n4

Congratulations! You just verified that the batch transform job is working as expected. Since the model is not required, you can delete it. Note that you are deleting the deployable model. Not the model package.

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

